In [1]:
import pandas as pd
import numpy as np

NUM_COURSES = 500
COURSES_PER_STUDENT = 7
NUM_STUDENTS = 500  # Assuming there are 500 students
NUM_DAYS = 21 
NUM_SLOTS = 5  

hall_capacities = {
    't1': 2000,
    't2': 2000,
    't3': 2000,
    't4': 2000,
    't5': 2000,
}

def generate_students(num_students, num_courses, courses_per_student):
    students_courses = {}
    all_courses = ['c' + str(i+1) for i in range(num_courses)]

    for student_id in range(1, num_students + 1):
        students_courses['s' + str(student_id)] = np.random.choice(all_courses, courses_per_student, replace=False)

    return students_courses

students_courses = generate_students(NUM_STUDENTS, NUM_COURSES, COURSES_PER_STUDENT)

def will_exceed_capacity(course, slot, cell_courses, students_courses, hall_capacities):
    count = 0  # Number of students taking the course at the same time slot
    for student, courses in students_courses.items():
        if course in courses and any(c in cell_courses for c in courses):
            count += 1
        # Check if the course being considered is already in the slot
        if course in courses and course not in cell_courses:
            count += 1
        # If the count exceeds capacity, return True
        if count > hall_capacities[slot]:
            return True
    return False

def clashes_with_students(course, cell_courses, students_courses):
    for student, courses in students_courses.items():
        if course in courses and any(c in courses for c in cell_courses):
            return True
    return False

def generate_individual(students_courses, num_days, num_slots, hall_capacities, all_courses):
    # Initialize empty timetable
    timetable = pd.DataFrame(index=['d' + str(i) for i in range(1, num_days+1)],
                             columns=['t' + str(i) for i in range(1, num_slots+1)], dtype=object)

    for row in timetable.index:
        for col in timetable.columns:
            timetable.at[row, col] = []  # Initialize each cell with an empty list

    # Shuffle the list of all courses to distribute them randomly across time slots
    shuffled_courses = np.random.permutation(all_courses)

    # Assign each course to a time slot, ensuring every course is placed at least once
    for course in shuffled_courses:
        placement_attempts = 0
        placed = False
        while not placed and placement_attempts < 200:
            # Select a random cell
            day, slot = np.random.choice(timetable.index), np.random.choice(timetable.columns)
            if (not will_exceed_capacity(course, slot, timetable.at[day, slot], students_courses, hall_capacities) and
                not clashes_with_students(course, timetable.at[day, slot], students_courses) and
                course not in timetable.at[day, slot]):
                timetable.at[day, slot].append(course)
                placed = True
            placement_attempts += 1

        if not placed:
            raise Exception(f"Unable to place course {course} without exceeding hall capacity or causing clashes after many attempts.")

    return timetable

def generate_population(num_individuals, students_courses, num_days, num_slots, hall_capacities, all_courses):
    return [generate_individual(students_courses, num_days, num_slots, hall_capacities, all_courses) for _ in range(num_individuals)]

# Usage
all_courses = ['c' + str(i+1) for i in range(NUM_COURSES)]
population = generate_population(5, students_courses, NUM_DAYS, NUM_SLOTS, hall_capacities, all_courses)


In [2]:
population

[                                                   t1  \
 d1                                   [c482, c9, c432]   
 d2          [c495, c16, c226, c162, c458, c213, c106]   
 d3               [c344, c163, c307, c363, c156, c223]   
 d4                                 [c349, c187, c380]   
 d5         [c392, c267, c253, c218, c381, c433, c415]   
 d6                                  [c499, c86, c417]   
 d7                     [c109, c325, c120, c146, c338]   
 d8                                   [c102, c8, c315]   
 d9                      [c210, c427, c63, c183, c428]   
 d10                                [c287, c362, c227]   
 d11                                 [c340, c21, c113]   
 d12                                  [c345, c225, c4]   
 d13                          [c308, c359, c245, c471]   
 d14                          [c370, c104, c318, c394]   
 d15                     [c369, c420, c121, c17, c485]   
 d16  [c263, c405, c256, c385, c346, c204, c130, c414]   
 d17          